In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import random
from joblib import Parallel, delayed
from scipy.interpolate import CubicSpline
from itertools import product

# Set random seeds for reproducibility
def set_random_seeds(seed_value=42):
    np.random.seed(seed_value)
    random.seed(seed_value)
    tf.random.set_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

set_random_seeds(42)

# Enable mixed precision if desired
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

tf.config.optimizer.set_experimental_options({
    'layout_optimizer': True,
    'constant_folding': True,
    'shape_optimization': True,
    'remapping': True,
    'arithmetic_optimization': True,
    'dependency_optimization': True,
    'loop_optimization': True,
    'function_optimization': True,
    'auto_mixed_precision': True,
    'auto_parallel': True
})

# Load the dataset
X = np.load('NEW_DATASET/dataset_20240709_X_3096.npy')
y = np.load('NEW_DATASET/dataset_20240709_y_3096.npy')

# Define the subject split points
split_60 = int(0.60 * len(X))
split_80 = int(0.80 * len(X))

# Data augmentation functions remain the same
def add_noise(data, noise_factor=0.0185):
    noise = np.random.randn(*data.shape) * noise_factor
    return np.clip(data + noise, 0, 1)

def scale(data, scale_factor=0.1605):
    scale = 1 + np.random.uniform(-scale_factor, scale_factor)
    return np.clip(data * scale, 0, 1)

def generate_random_curves(data, sigma=0.078, knot=2):
    xx = np.linspace(0, data.shape[0] - 1, knot + 2)
    yy = np.random.normal(loc=1.0, scale=sigma, size=(knot + 2, data.shape[1]))
    x_range = np.arange(data.shape[0])
    augmented_data = np.zeros_like(data)
    for i in range(data.shape[1]):
        cs = CubicSpline(xx, yy[:, i])
        augmented_data[:, i] = data[:, i] * cs(x_range)
    return np.clip(augmented_data, 0, 1)

def augment_sample(sample):
    sample = add_noise(sample)
    sample = scale(sample)
    sample = generate_random_curves(sample)
    return sample

# Adjusted augment_dataset function remains the same
def augment_dataset(X, y, target_size=10000):
    num_original = len(X)
    num_augmented = target_size - num_original

    indices = np.random.randint(0, num_original, size=num_augmented)

    def process_sample(idx):
        augmented_x = augment_sample(X[idx])
        return augmented_x, y[idx]

    augmented_samples = Parallel(n_jobs=-1)(
        delayed(process_sample)(idx) for idx in indices
    )

    augmented_X = np.array([sample[0] for sample in augmented_samples])
    augmented_y = np.array([sample[1] for sample in augmented_samples])
    return np.vstack((X, augmented_X)), np.vstack((y, augmented_y))

# Create data folder if it doesn't exist
data_folder = 'processed_datasets'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

# Split indices for the subjects
split_60 = int(0.60 * len(X))
split_80 = int(0.80 * len(X))

# Function to load or create datasets for each subject remains the same
def load_or_create_subject_datasets(X, y, subject_number, split_indices, data_folder):
    # Determine file paths
    X_original_path = os.path.join(data_folder, f'X_subject{subject_number}_original.npy')
    y_original_path = os.path.join(data_folder, f'y_subject{subject_number}_original.npy')
    X_aug_path = os.path.join(data_folder, f'X_subject{subject_number}_aug.npy')
    y_aug_path = os.path.join(data_folder, f'y_subject{subject_number}_aug.npy')

    # Check if files exist
    files_exist = all(os.path.exists(p) for p in [X_original_path, y_original_path, X_aug_path, y_aug_path])

    if files_exist:
        print(f"Loading datasets for Subject {subject_number} from disk.")
        # Load datasets
        X_subject_original = np.load(X_original_path)
        y_subject_original = np.load(y_original_path)
        X_subject_aug = np.load(X_aug_path)
        y_subject_aug = np.load(y_aug_path)
    else:
        print(f"Creating and saving datasets for Subject {subject_number}.")
        # Split data for the subject
        start_idx, end_idx = split_indices
        X_subject_original = X[start_idx:end_idx]
        y_subject_original = y[start_idx:end_idx]
        # Save original data
        np.save(X_original_path, X_subject_original)
        np.save(y_original_path, y_subject_original)
        # Augment data
        X_subject_aug, y_subject_aug = augment_dataset(X_subject_original, y_subject_original, 10000)
        # Save augmented data
        np.save(X_aug_path, X_subject_aug)
        np.save(y_aug_path, y_subject_aug)

    return (X_subject_original, y_subject_original), (X_subject_aug, y_subject_aug)

# Load or create datasets for subjects
(X_subject1, y_subject1), (X_subject1_aug, y_subject1_aug) = load_or_create_subject_datasets(
    X, y, subject_number=1, split_indices=(0, split_60), data_folder=data_folder)

(X_subject2, y_subject2), (X_subject2_aug, y_subject2_aug) = load_or_create_subject_datasets(
    X, y, subject_number=2, split_indices=(split_60, split_80), data_folder=data_folder)

(X_subject3, y_subject3), (X_subject3_aug, y_subject3_aug) = load_or_create_subject_datasets(
    X, y, subject_number=3, split_indices=(split_80, len(X)), data_folder=data_folder)

# Store datasets in a dictionary remains the same
subject_datasets = {
    1: {"aug": (X_subject1_aug, y_subject1_aug), "original": (X_subject1, y_subject1)},
    2: {"aug": (X_subject2_aug, y_subject2_aug), "original": (X_subject2, y_subject2)},
    3: {"aug": (X_subject3_aug, y_subject3_aug), "original": (X_subject3, y_subject3)}
}

# Prepare datasets for given train/test subject combinations remains the same
def prepare_datasets(train_subjects, test_subject):
    X_train = np.vstack([subject_datasets[train]["aug"][0] for train in train_subjects])
    y_train = np.vstack([subject_datasets[train]["aug"][1] for train in train_subjects])
    X_test, y_test = subject_datasets[test_subject]["original"]
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
    return X_train, y_train, X_val, y_val, X_test, y_test

# Bi-LSTM model
def build_bilstm_model(input_shape, lstm_units=32, dropout_rate=0.0):
    model = tf.keras.Sequential([
        layers.Bidirectional(layers.LSTM(lstm_units, return_sequences=False), input_shape=input_shape),
        layers.Dropout(dropout_rate),
        layers.Dense(64, activation='gelu'),
        layers.Dropout(dropout_rate),
        layers.Dense(9, activation='softmax', dtype='float32')
    ])
    return model

# Custom learning rate scheduler with warmup and linear decay
class WarmupDecayLearningRateScheduler(tf.keras.callbacks.Callback):
    def __init__(self, initial_lr, warmup_epochs, total_epochs, min_lr=1e-8):
        super().__init__()
        self.initial_lr = initial_lr
        self.warmup_epochs = warmup_epochs
        self.total_epochs = total_epochs
        self.min_lr = min_lr

    def on_epoch_begin(self, epoch, logs=None):
        if epoch < self.warmup_epochs:
            lr = (self.initial_lr / self.warmup_epochs) * (epoch + 1)
        else:
            decay_steps = self.total_epochs - self.warmup_epochs
            lr = self.initial_lr * max((decay_steps - (epoch - self.warmup_epochs)) / decay_steps, self.min_lr)
        tf.keras.backend.set_value(self.model.optimizer.lr, lr)

def train_and_evaluate(X_train, y_train, X_val, y_val, X_test, y_test, hyperparameters):
    learning_rate = hyperparameters['learning_rate']
    model = build_bilstm_model(input_shape=(30, 126), lstm_units=hyperparameters['lstm_units'], dropout_rate=hyperparameters['dropout_rate'])

    optimizer = mixed_precision.LossScaleOptimizer(tf.keras.optimizers.Adam(learning_rate=learning_rate))
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["categorical_accuracy"])

    batch_size = 2048  # Adjustable batch size

    # Create TensorFlow datasets with adjustable batch size
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    # Learning rate scheduler and early stopping
    lr_scheduler = WarmupDecayLearningRateScheduler(initial_lr=learning_rate, warmup_epochs=20, total_epochs=200)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=200, restore_best_weights=True)

    history = model.fit(
        train_dataset, epochs=200, validation_data=val_dataset, 
        callbacks=[lr_scheduler, early_stopping], verbose=0
    )

    train_acc = history.history['categorical_accuracy'][-1]
    val_acc = history.history['val_categorical_accuracy'][-1]
    test_loss, test_acc = model.evaluate(test_dataset, verbose=0)

    y_pred = np.argmax(model.predict(X_test), axis=1)
    y_true = np.argmax(y_test, axis=1)
    f1 = classification_report(y_true, y_pred, output_dict=True)['weighted avg']['f1-score']

    print(f"Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}, F1: {f1:.4f}")
    return train_acc, val_acc, test_acc, f1

def run_experiments():
    lstm_units_list = [64, 128, 256]
    dropout_rates = [0.1,0.2]
    learning_rates = np.linspace(1e-2, 1e-4, 5)

    for lstm_units, dropout_rate, learning_rate in product(lstm_units_list, dropout_rates, learning_rates):
        hyperparameters = {'lstm_units': lstm_units, 'dropout_rate': dropout_rate, 'learning_rate': learning_rate}
        print(f"\nEvaluating Hyperparameters: {hyperparameters}")

        results = []
        for train_subjects, test_subject in [([1, 2], 3), ([1, 3], 2), ([2, 3], 1)]:
            print(f"Training on Subjects {train_subjects}, Testing on Subject {test_subject}")
            X_train, y_train, X_val, y_val, X_test, y_test = prepare_datasets(train_subjects, test_subject)
            results.append(train_and_evaluate(X_train, y_train, X_val, y_val, X_test, y_test, hyperparameters))

        avg_train_acc = np.mean([res[0] for res in results])
        avg_val_acc = np.mean([res[1] for res in results])
        avg_test_acc = np.mean([res[2] for res in results])
        avg_f1 = np.mean([res[3] for res in results])

        print(f"Average Train Acc: {avg_train_acc:.4f}, Average Val Acc: {avg_val_acc:.4f}, Average Test Acc: {avg_test_acc:.4f}, Average F1: {avg_f1:.4f}")

run_experiments()


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6
Loading datasets for Subject 1 from disk.
Loading datasets for Subject 2 from disk.
Loading datasets for Subject 3 from disk.

Evaluating Hyperparameters: {'lstm_units': 64, 'dropout_rate': 0.1, 'learning_rate': 0.01}
Training on Subjects [1, 2], Testing on Subject 3
10/10 [==============================] - 1s 5ms/step
Train Acc: 0.9999, Val Acc: 0.8839, Test Acc: 0.8839, F1: 0.8885
Training on Subjects [1, 3], Testing on Subject 2
10/10 [==============================] - 1s 4ms/step
Train Acc: 0.9999, Val Acc: 0.6246, Test Acc: 0.6935, F1: 0.6879
Training on Subjects [2, 3], Testing on Subject 1
30/30 [==============================] - 1s 3ms/step
Train Acc: 0.9999, Val Acc: 0.7252, Test Acc: 0.7083, F1: 0.7092
Average Train Acc: 0.9999, Average Va